# The PDF parser


**Goal:** Extract clean text and page numbers from a user-uploaded PDF.

In [3]:
import fitz  # Import the PyMuPDF library

In [4]:
# Define a function to accept pdf with its path
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path) # Open the Document
    # Preparing "Collection Basket"
    pages_data = [] 
    # loop through each page in the doc
    for page_no , page in enumerate(doc , start=1):
        texts = page.get_text()
        page_dict = {"page_no":page_no , "text":texts}
        pages_data.append(page_dict)

    return pages_data        